In [34]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from datetime import date, timedelta
from time import time
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults

<ipython-input-34-2d6a47e37b07>:6: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [35]:
transaction_data = pd.read_csv('../datasets/transaction_data.tsv', sep='\t')
transaction_data

,date,TICKER,COMNAM,BIDLO,ASKHI,OPENPRC,PRC,VOL,SHROUT
0,2012/01/03,ORCL,ORACLE CORP,25.8300,26.66,26.33,25.865,38081439.0,5025837.0
1,2012/01/04,ORCL,ORACLE CORP,25.5500,26.23,25.95,26.010,45599113.0,5025837.0
2,2012/01/05,ORCL,ORACLE CORP,25.9100,26.90,26.01,26.590,51020071.0,5025837.0
3,2012/01/06,ORCL,ORACLE CORP,26.3600,27.15,26.67,26.930,55830810.0,5025837.0
4,2012/01/09,ORCL,ORACLE CORP,26.6600,27.12,26.90,27.030,38596800.0,5025837.0
...,...,...,...,...,...,...,...,...,...
105010,2012/12/24,CBOE,C B O E HOLDINGS INC,29.7150,30.16,30.12,29.880,285966.0,87272.0
105011,2012/12/26,CBOE,C B O E HOLDINGS INC,29.1400,29.77,29.77,29.580,557886.0,87272.0
105012,2012/12/27,CBOE,C B O E HOLDINGS INC,29.1600,29.64,29.64,29.330,563717.0,87272.0
105013,2012/12/28,CBOE,C B O E HOLDINGS INC,29.1101,29.53,29.27,29.370,1055427.0,87272.0


In [44]:
stocks = {}
for ticker in transaction_data['TICKER'].unique():
    stocks[ticker] = transaction_data[transaction_data['TICKER'] == ticker]

In [73]:
def get_arima_prediction(ticker, future_observations_count = 100):
    start = time()
    ticker_data = stocks[ticker]
    
    dates = ticker_data['date'].values.tolist()
    df = ticker_data['PRC'].values
    
    last_date_string = dates[-1]
    last_date_split = [int(i) for i in last_date_string.split('/')]
    last_date = date(*last_date_split)
    
    
    history = [x for x in df]
    is_prediction = [True for i in history]
    model_predictions = []
    
    for time_point in range(future_observations_count):
        if(last_date.weekday() == 4):
            last_date += timedelta(days=3)
        else:
            last_date += timedelta(days=1)
            
        model = ARIMA(history, order=(7,1,0))
        model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        y_hat = output[0]
        model_predictions.append(y_hat)
        history.append(y_hat[0])
        dates.append(last_date.strftime("%Y/%m/%d"))
        is_prediction.append(False)
    
    print(f"{time()-start} s")

    return dates, history, is_prediction

In [77]:
i = 1
for ticker in list(stocks.keys())[209:]:
    print(f"{i}. {ticker} ", end="")
    try:
        dates, history, is_prediction = get_arima_prediction(ticker, 50)

        df = pd.DataFrame({'date': dates, 'PRC': history, 'is_prediction': is_prediction }, columns=['date', 'PRC', 'is_prediction'])
        df.to_csv (f'../database/arima/{ticker}.csv', index = False, header=True)
    except:
        pass
    i += 1

1. PNR 2. TMO 13.14691710472107 s
3. CSX 12.277730941772461 s
4. TMK 11.972368001937866 s
5. CI 15.951825141906738 s
6. NSC 11.231620073318481 s
7. PGR 12.061743259429932 s
8. STZ 16.787381887435913 s
9. D 11.643258810043335 s
10. KEY 21.986616849899292 s
11. AOS 15.058452129364014 s
12. COO 10.087332248687744 s
13. VZ 16.603970050811768 s
14. T 17.110431909561157 s
15. USB 12.05617904663086 s
16. HD 16.01871395111084 s
17. AIG 13.402752876281738 s
18. STI 14.576038122177124 s
19. MS 12.94374704360962 s
20. MYL 14.227904081344604 s
21. RJF 13.197730779647827 s
22. HOG 10.1456937789917 s
23. APC 10.663284063339233 s
24. C 12.889395952224731 s
25. ECL 14.746920108795166 s
26. UNM 11.8628511428833 s
27. BBT 15.937469005584717 s
28. STT 14.107834100723267 s
29. SYK 13.68857192993164 s
30. TIF 11.170747756958008 s
31. CBS 14.039880275726318 s
32. SCHW 11.21628713607788 s
33. PXD 9.990583658218384 s
34. ADBE 15.105762004852295 s
35. SYMC 14.654623985290527 s
36. AGN 10.88687515258789 s
37. E

/Users/welvin/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "


9.63130807876587 s
53. ROP 11.446072101593018 s
54. SNPS 11.6126868724823 s
55. TWX 13.612658739089966 s
56. M 16.460297107696533 s
57. MHK 10.237123966217041 s
58. BSX 22.495317935943604 s
59. KSS 12.327924251556396 s
60. DHI 11.323245763778687 s
61. ESRX 9.839441061019897 s
62. SBUX 11.147548913955688 s
63. TSN 16.486634016036987 s
64. PX 11.066486835479736 s
65. PDCO 11.726519107818604 s
66. CHK 15.098616123199463 s
67. INTU 15.031333208084106 s
68. MCHP 12.582251071929932 s
69. ORLY 9.921010971069336 s
70. FLIR 15.804800033569336 s
71. ALL 16.238683938980103 s
72. BWA 10.074790000915527 s
73. UHS 12.19105577468872 s
74. ATVI 17.2096529006958 s
75. IT 

/Users/welvin/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "


11.26253890991211 s
76. INCY 12.636779069900513 s
77. NFX 10.439221143722534 s
78. EMN 11.605318069458008 s
79. ALB 10.938272953033447 s
80. MLM 9.713653087615967 s
81. TSCO 10.026525974273682 s
82. COF 12.853339910507202 s
83. MCK 10.733740091323853 s
84. DLTR 9.729203939437866 s
85. ABC 11.39877200126648 s
86. DRI 10.41903805732727 s
87. DISH 13.726266145706177 s
88. RMD 11.933742046356201 s
89. FCX 11.930188179016113 s
90. DVA 9.872631072998047 s
91. HSIC 13.070451021194458 s
92. NTAP 10.952210903167725 s
93. EL 9.343775033950806 s
94. WAT 10.750685930252075 s
95. CTXS 9.946244716644287 s
96. HIG 16.361847162246704 s
97. ALXN 9.451774835586548 s
98. IRM 13.95642614364624 s
99. ANSS 10.045713901519775 s
100. ETFC 

/Users/welvin/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "


11.528474807739258 s
101. SRCL 11.021756887435913 s
102. NOV 

/Users/welvin/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "


9.690474271774292 s
103. ZION 12.68677306175232 s
104. DGX 13.001398086547852 s
105. ROK 11.68997597694397 s
106. APH 11.14978313446045 s
107. AMZN 10.072890281677246 s
108. RL 9.70253610610962 s
109. AME 10.183706998825073 s
110. VLO 13.593626022338867 s
111. YUM 11.51033091545105 s
112. CHRW 12.855167150497437 s
113. AMG 11.315616846084595 s
114. MTD 10.514388799667358 s
115. ADSK 10.269252061843872 s
116. URI 10.884192943572998 s
117. VRSN 10.665849208831787 s
118. PWR 11.562681913375854 s
119. MAR 15.662544965744019 s
120. BBY 14.881799936294556 s
121. SEE 11.392675876617432 s
122. LLL 14.193234205245972 s
123. SRE 11.162304162979126 s
124. CTSH 10.231346130371094 s
125. RSG 16.63682723045349 s
126. CCI 16.354342937469482 s
127. EBAY 14.328508853912354 s
128. NVDA 11.946578979492188 s
129. PCLN 13.46541714668274 s
130. GS 9.630491971969604 s
131. FFIV 9.912842988967896 s
132. JNPR 13.847190856933594 s
133. SBAC 15.270697116851807 s
134. COST 12.566503047943115 s
135. DVN 10.9686369